## Park-UT: An app to help drivers find a spot to park at the University of Texas

#### Brief description:
This notebook is to serve as representative of the work done for the MIS 382N (69515) course project 1.

Park-UT lets a user reserve a spot to park in a parking garage. The user may then park at the reserved spot.

#### Assumptions:
The app defines a venue as parking spots at garages. It also defines an event as a reservation made at that spot.

### Setup and utility functions

In [2]:
import sqlite3
import random
from datetime import datetime, timedelta
import re

In [3]:
db=sqlite3.connect("./db/UTPark.db")
cursor = db.cursor()

In [4]:
# turn this on if you want more details of what the functions are doing
_debug = False

In [5]:
def debug(output):
    if _debug:
        print('DEBUG: ' + output)

def string_to_date(string):
    date = datetime.fromisoformat(string)
    debug('string_to_date - {} converted to {}'.format(string, date))
    return date

def row_to_user (row):
    user = {}
    user['id'] = row[0]
    user['name'] = row[1]
    user['email'] = row[2]
    user['admin'] = bool(row[3])
    debug('row_to_user - {} converted to {}'.format(row, user))
    return user

def row_to_spot (row):
    spot = {}
    spot['id'] = row[0]
    spot['garageid'] = row[1]
    spot['level'] = row[2]
    spot['number'] = row[3]
    debug('row_to_spot - {} converted to {}'.format(row, spot))
    return spot

def row_to_reservation(row):
    res = {}
    res['id'] = row[0]
    res['spotid'] = row[1]
    res['expiry'] = string_to_date(row[2])
    debug('row_to_reservation - {} converted to {}'.format(row, res))
    return res

def row_to_occupancy(row):
    occ = {}
    occ['id'] = row[0]
    occ['spotid'] = row[1]
    occ['start'] = string_to_date(row[2])
    debug('row_to_occupancy - {} converted to {}'.format(row, occ))
    return occ

def fetch_all():
    all = cursor.fetchall()
    for pl in all:
        return(pl)


In [6]:
def get_spots(garageid):
    # utility method that returns a generator function that gets spots at a garage
    cursor.execute('''SELECT * FROM spots WHERE GarageId={}'''.format(garageid))
    return (row_to_spot(row) for row in cursor.fetchall())

In [17]:
def get_reservation(spotid):
    # utility method that serves as a mix of a reservation pruner and fetcher.
    # more efficient to have everything that accesses a reservation go through here
    # than implement a background process or something similar to clear expired reservations
    cursor.execute('''SELECT * FROM reservations WHERE SpotId={}'''.format(spotid))
    reservation = cursor.fetchone()
    if reservation is None:
        debug('get_reservation - No reservation found.')
        return None
    
    reservation = row_to_reservation(reservation)
    
    if reservation['expiry'] < datetime.now():
        cursor.execute('''DELETE FROM reservations WHERE SpotId={}'''.format(spotid))
        debug('get_reservation - Expired reservation found. Deleted.')
        db.commit()
        return None
    else:
        debug('get_reservation - Returning {}'.format(reservation))
        return reservation

print(get_reservation(1))
print(get_reservation(2))

None
None


In [21]:
def get_occupancy(spotid):
    # utility method that serves as a occupancy fetcher
    cursor.execute('''SELECT * FROM occupancies WHERE SpotId={}'''.format(spotid))
    occupancy = cursor.fetchone()
    if occupancy is None:
        debug('get_occupancy - No occupancy found.')
        return None
    else:
        occupancy = row_to_occupancy(occupancy)
        debug('get_occupancy - Returning {}'.format(occupancy))
        return occupancy
    
print(get_occupancy(22))
print(get_occupancy(23))

None
None


In [7]:
admin = None
non_admin = None
cursor.execute('''SELECT * FROM users''')
for pl in cursor.fetchall():
    debug(pl)
    if non_admin is None and pl[3] is 0:
        non_admin = row_to_user(pl)
    elif admin is None and pl[3] is 1:
        admin = row_to_user(pl)

print('non_admin: ', non_admin)
print('admin: ', admin)

non_admin:  {'id': 2, 'name': 'Catherine', 'email': 'cathy.morgan@utexas.edu', 'admin': False}
admin:  {'id': 1, 'name': 'Vrinda', 'email': 'vrinda.sha@utexas.edu', 'admin': True}


###  1. Add a new user (admin only)

In [8]:
def add_user(user, name, email, is_admin):
    # add new user
    if not (user['admin']):
        print('Insufficient access.')
        return
    
    try:
        insertsql = '''INSERT INTO users (Name,Email,Admin) VALUES ('{}', '{}', {})'''
        cursor.execute(insertsql.format(name, email, 1 if is_admin is True else False))
        print('Successfully added a user.')
        db.commit()
    except Exception as e:
        print(e)

add_user(non_admin, 'Zara', 'zara@gmail.com', False)
add_user(admin, 'Zara', 'zara@gmail.com', False)

Insufficient access.
UNIQUE constraint failed: users.Email


###  2. Add a new venue (admin only)

We interpret a venue to be a garage or a spot.

In [9]:
def add_garages(user, name, address):
    if not (user['admin']):
        print('Insufficient access.')
        return
    
    try:
        #Checking if the garage already exists
        #and avoiding redundant entries to garages table
        cursor.execute('''Select name from garages''')
        existing_garages = cursor.fetchall()
        
        current_garage = name,
        print()
        if current_garage in existing_garages:
            print(current_garage[0],'--> This Garage already exists!')
            cursor.execute('''SELECT Name FROM garages''')
            print()
            print('Existing Garages:')
            print()
            all=cursor.fetchall()
            for pl in all:
                print(pl[0])
            return
        else:    
            insertsql = '''INSERT INTO garages (Name, Address) VALUES ('{}', '{}')'''
            cursor.execute(insertsql.format(name, address))
            print('Successfully added a garage.')
        db.commit()
    except Exception as e:
        print(e)

add_garages(non_admin, 'Brazos Garages (BRG)', '210 E. MLK Blvd.')
add_garages(admin, 'Brazos Garages (BRG)', '210 E. MLK Blvd.')

Insufficient access.

Brazos Garages (BRG) --> This Garage already exists!

Existing Garages:

Brazos Garages (BRG)
Conference Center Garage (CCG)
East Campus Garage (ECG)
Health Center Garage (HCG)
Manor Garage (MAG)
Trinity Garage (TRG)
Speedway Garage (SWG)
San Antonio Garage (SAG)
Guadalupe Garage (GUG)
Rowling Hall Garage (RHG)


In [10]:
def add_spots(user, garage_id, level):
    if not (user['admin']):
        print('Insufficient access.')
        print()
        return
    
    try:
        #Checking if the garage exists
        #Providing the list of available garages to add a spot
        cursor.execute('''Select count(*) from garages where Id={}'''.format(garage_id))
        count = cursor.fetchone()
        if count[0] is 0:
            print('You are trying to add a spot to a Garage that does not exist!')
            cursor.execute('''SELECT Id FROM garages''')
            all = cursor.fetchall()
            print()
            print('Choose a different GarageId from below :-')
            for pl in all:
                
                print(pl[0])
            
        else:
            #Adding new spot only if the garage exists
            insertsql = '''INSERT INTO spots (GarageId, Level, Number) VALUES ({}, {}, {})'''
            cursor.execute(insertsql.format(garage_id, level, random.randint(1,10000)))
            print('Successfully added a spot.')

        db.commit()
    except Exception as e:
        print(e)

add_spots(non_admin, 22, 1)
add_spots(admin, 22, 1)

Insufficient access.

Successfully added a spot.


###  3. Start an event (user or admin on behalf of a user)

We interpret starting an event to be making a reservation.

In [54]:
def make_reservation(user, spotid, onbehalfof=None):
    # this reserves a spot for 10 minutes
    if onbehalfof is not None:
        if not (user['admin']):
            print('Insufficient access to reserve on behalf of another user.')
            return
        user = onbehalfof
    
    try:      
        cursor.execute('''SELECT count(*) FROM spots WHERE Id={}'''.format(spotid))
        count = cursor.fetchone()
        
        if count[0] is 0:
            print(spotid,' - Spot does not exist!')
        else:
            
            #first, check if the spot is already occupied
            cursor.execute('''SELECT count(*) FROM occupancies WHERE SpotId={}'''.format(spotid))
            is_occ = cursor.fetchone()
            if is_occ[0] != 0:
                print('This spot is currently occupied.')
                return

            #second, check if the spot is already reserved
            existing = get_reservation(spotid)
            if existing is not None:
                print('This spot is already reserved.')
                return

            #third, make the reservation
            expire = (datetime.now() + timedelta(minutes=10)).isoformat(timespec='minutes')
            insertsql = '''INSERT INTO reservations (SpotId, Expiry) VALUES ({}, '{}')'''
            cursor.execute(insertsql.format(spotid, expire))

            print('Successfully reserved the spot till', expire)

        db.commit()
    except Exception as e:
        print(e)

# should pass
make_reservation(non_admin, 67)

# should fail because a non-admin cannot add reserve on behalf of another user
make_reservation(non_admin, 28, non_admin)

# should fail because although the admin can do this, a reservation would already exist
# see function 9 to kill the reservation
make_reservation(admin, 31, non_admin)

67  - Spot does not exist!
Insufficient access to reserve on behalf of another user.
Successfully reserved the spot till 2019-07-28T18:48


### 4. Display timeslot availability at a venue

We interpret this to be "Display if a spot is available at a garage".

In [39]:
def is_spot_available(spotid):
    # returns if a spot is available at a garage
    reservation = get_reservation(spotid)
    if reservation is None:
        occupancy = get_occupancy(spotid)
        if occupancy is None:
            debug('This spot is available.')
            return True
    
    debug('This spot is unavailable.')
    return False

print(is_spot_available(27))
print(is_spot_available(29))

False
False


### 5. Display all venues where a particular timeslot is available

Our system discards reservations that have expired; all reservations that exist are current.
Therefore, we interpret this to be "Display all slots in a garage that are available".

In [40]:
def get_available_spots(garageid):
    # returns list of all slots in a garage that are available
    cursor.execute('''SELECT * FROM spots WHERE garageid={}'''.format(garageid))
    return [spot['number'] for spot in (row_to_spot(row) for row in cursor.fetchall()) if is_spot_available(spot['id'])]

print(get_available_spots(22))
print(get_available_spots(23))

[6577, 9443, 4205, 3860, 7224]
[3935, 5265]


### 6. List events at a venue given date/time

We have to interpret this to be "List all reservations at a garage"

In [41]:
def get_reserved_spots(garageid):
    return [res for res in (get_reservation(spot['id']) for spot in get_spots(garageid)) if res is not None]
    
get_reserved_spots(22)
get_reserved_spots(23)

[]

###  7. User joins an event

We interpret this to be "User occupies a reserved spot".

Our business rules here are that a user cannot occupy a spot without a prior reservation and that a user cannot occupy a spot that is already occupied.

In [45]:
def occupy_reservation(spotid):
    # user occupies a reserved spot
    
    try:
        cursor.execute('''SELECT count(*) FROM spots WHERE Id={}'''.format(spotid))
        count = cursor.fetchone()
        
        if count[0] is 0:
            print(spotid,' - Spot does not exist!')
        else:
            #first, check if spot is occupied
            cursor.execute('''SELECT * FROM occupancies WHERE SpotId={}'''.format(spotid))
            existingoccupancy = cursor.fetchone()
            if existingoccupancy is not None:
                print('This spot is already occupied.')
                return

            #second, check if the reservation exists
            reservation = get_reservation(spotid)
            if reservation is None:
                print('No reservation exists for this spot.')
                return

            #third, make occupancy
            insertsql = '''INSERT INTO occupancies (SpotId, StartTime) VALUES ({}, '{}')'''
            cursor.execute(insertsql.format(spotid, datetime.now()))
            print('Awesome! We are glad to have you here!')

            #fourth, delete reservation
            deleteres = '''DELETE FROM reservations WHERE SpotId={}'''.format(spotid)
            cursor.execute(deleteres)

            db.commit()
        
    except Exception as e:
        db.rollback()
        print(e)

occupy_reservation(27)
occupy_reservation(29)

This spot is already occupied.
This spot is already occupied.


In [46]:
def end_occupancy(spotid):
    cursor.execute('''SELECT * FROM occupancies where SpotId={}'''.format(spotid))
    occupancy = cursor.fetchone()
    if occupancy is None:
        print('The spot is not occupied.')
    else:
        cursor.execute('''DELETE FROM occupancies where SpotId={}'''.format(spotid))
        print('The spot is no longer occupied.')
        db.commit()

end_occupancy(27)
end_occupancy(29)

The spot is no longer occupied.
The spot is no longer occupied.


###  9. Remove an event (admin only)

We interpret an event here to be a spot reservation.

In [49]:
def remove_reservation(user, spotid):
    if not (user['admin']):
        print('Insufficient access to remove a reservation.')
        return
    
    if get_reservation(spotid) is None:
        print('The spot is not reserved.')
    else:
        cursor.execute('''DELETE FROM reservations where SpotId={}'''.format(spotid))
        print('The spot is no longer reserved.')
        db.commit()

remove_reservation(admin, 27)
remove_reservation(non_admin, 29)

The spot is not reserved.
Insufficient access to remove a reservation.


In [49]:
db.close()